In [2]:
#Open relevant packages
import requests
import argparse
import sys
import json
import pandas as pd
import numpy as np
from pprint import pprint
import re

In [3]:
#Define arguments
#Reference - https://github.com/Milchdealer/isoform-lister/blob/main/main.py 
sys.argv = [''] #to avoid unrecognised arguments error
parse = argparse.ArgumentParser()
#Adding arguments
parse.add_argument(
        "--searchlist",
        default="glyco_unip.csv",
)
parse.add_argument(
        "--output",
        default="glyco_unip_part1.csv",
)

_StoreAction(option_strings=['--output'], dest='output', nargs=None, const=None, default='glyco_unip_part1.csv', type=None, choices=None, help=None, metavar=None)

In [111]:
un = parse.parse_args() #Parse arguments
terms = []
# Creating a list of UniProt IDs
with open(un.searchlist) as l:
    for line in l:
        terms.append(line)
fi = ['gene_orf', 'gene_primary', 'protein_name', 'ec', 'go_c', 'length', 'xref_alphafolddb']
a = ",".join(fi) #https://stackoverflow.com/questions/5299796/how-to-use-list-or-tuple-as-string-formatting-value
url = "https://rest.uniprot.org/uniprotkb/search?query=accession:{0}&fields={1}"
uni = []
for term in terms:
    results = requests.get(url.format(term, a)).text #Converting response into text
    data = json.loads(results) #To convert json string to dictionary; Reference - https://towardsdatascience.com/json-and-apis-with-python-fba329ef6ef0
    uni.extend(data['results']) #To add looped results into list

In [1]:
#pprint(uni)

Pretty printing has been turned OFF


In [112]:
#List comprehension for extracting UniProt IDs
ids = [item["primaryAccession"] for item in uni]
print(ids)

['O15648', 'Q38B42', 'Q586Z6', 'Q9NJP9', 'Q583Z3', 'Q57Y11', 'Q57ZS7', 'Q586V2', 'Q57YL8', 'Q385E1', 'Q585R0', 'Q38AC3', 'Q387E8', 'Q57YS3', 'Q387X2', 'Q57U85', 'Q385L6', 'Q38EA4', 'Q57YW0', 'Q383C6', 'Q582I3']


In [113]:
# A function to extract nested dictionaries from the main list
def findkeys(node, kv): #https://stackoverflow.com/questions/9807634/find-all-occurrences-of-a-key-in-nested-dictionaries-and-lists
    if isinstance(node, list):
        for i in node:
            for x in findkeys(i, kv):
                yield x
    elif isinstance(node, dict):
        if kv in node:
            yield node[kv]
        for j in node.values():
            for x in findkeys(j, kv):
                yield x

In [114]:
# A function to extract values based on keys
def find(key, dictionary): #https://gist.github.com/douglasmiranda/5127251
    for k, v in dictionary.items():
        if k == key:
            yield v
        elif isinstance(v, dict):
            for result in find(key, v):
                yield result
        elif isinstance(v, list):
            for d in v:
                for result in find(key, d):
                    yield result

In [115]:
woohoo = []
for j in range(len(uni)):
    yippee = list(find('value', uni[j]))
    woohoo.append(yippee[0])
woohoo

['ATP-dependent 6-phosphofructokinase',
 'Fructose-bisphosphate aldolase',
 'Glyceraldehyde-3-phosphate dehydrogenase',
 'Glycerol kinase, glycosomal',
 'Dihydroxyacetone phosphate acyltransferase, putative',
 'Mevalonate kinase',
 "Inosine-5'-monophosphate dehydrogenase",
 'Phosphoenolpyruvate carboxykinase (ATP)',
 'Transketolase',
 'Pyruvate, phosphate dikinase',
 'Alkyl-dihydroxyacetone phosphate synthase',
 'Uncharacterized protein',
 'Ribokinase',
 'Inositol polyphosphate 1-phosphatase, putative',
 'Malate dehydrogenase',
 'Orotidine-5-phosphate decarboxylase/orotate phosphoribosyltransferase, putative',
 'Triosephosphate isomerase',
 'Fructose-bisphosphatase',
 'Glycerol-3-phosphate dehydrogenase [NAD(+)]',
 'Glycosomal membrane protein, putative',
 'Deoxyribose-phosphate aldolase']

In [116]:
locs = list(findkeys(uni, 'uniProtKBCrossReferences'))
#print(locs)
c_locss = []
for i in range(len(uni)):
    luca = ",".join(j for l in locs[i] for j in l['properties'][0]['value'].splitlines() if j.startswith('C:'))
    c_locss.append(luca)
    
    
        
            

    
c_locss

['C:glycosome',
 'C:cytosol',
 'C:cytosol,C:glycosome',
 'C:glycosome',
 'C:acidocalcisome,C:cytoplasm,C:glycosome,C:mitochondrial membrane',
 'C:cytosol,C:glycosome',
 'C:cytoplasm,C:glycosome,C:nucleolus',
 'C:contractile vacuole,C:cytosol,C:glycosome,C:vesicle membrane',
 'C:ciliary plasm,C:cytoplasm,C:cytosol,C:glycosome,C:nuclear lumen,C:nucleus',
 'C:glycosome',
 'C:glycosome,C:glycosome lumen',
 'C:glycosome',
 'C:glycosome,C:nucleus',
 'C:glycosome',
 'C:ciliary plasm,C:cytoplasm,C:glycosome',
 'C:glycosome',
 'C:cytosol',
 'C:cytoplasm,C:cytosol,C:glycosome',
 'C:cytosol,C:glycerol-3-phosphate dehydrogenase complex,C:glycosome',
 'C:endoplasmic reticulum,C:glycosome,C:glycosome membrane,C:integral component of peroxisomal membrane,C:peroxisomal membrane',
 'C:glycosome']

In [117]:
ecnum = list(findkeys(uni, 'ecNumbers'))
#print(ecnum)
ec_values = list(findkeys(ecnum, 'value'))
#print(ec_values)

In [118]:
woohoo2 = []
for j in range(len(uni)):
    yippee = list(find('value', uni[j]))
    woohoo2.append(yippee[2])
#print(woohoo2)
new = ["pfk", "GK", "Tb927.4.4070", "Tb10.6k15.2070", "Tb11.03.0090", "Tb927.5.3810", "Tb11.01.3370", "Tb927.7.5680"]
positions = [0,3,5,11,12,15,19,20]
for (index, replacement) in zip(positions, new):
    woohoo2[index] = replacement      
print(woohoo2)

['pfk', 'Tb10.70.1370', 'Tb927.6.4300', 'GK', 'Tb927.4.3160', 'Tb927.4.4070', 'Tb927.5.2080', 'Tb927.2.4210', 'Tb927.8.6170', 'Tb11.02.4150', 'Tb927.6.1500', 'Tb10.6k15.2070', 'Tb11.03.0090', 'Tb927.8.7170', 'Tb10.61.0980', 'Tb927.5.3810', 'Tb11.02.3210', 'Tb09.211.0540', 'Tb927.8.3530', 'Tb11.01.3370', 'Tb927.7.5680']


In [119]:
seq_length = [it["sequence"]["length"] for it in uni]
#print(seq_length)

In [120]:
# reso = list(iterate_all(uni))
# #print(reso)
# values = []
# for line in reso:
#     newList = re.findall(r"^\d*\.\d\d.\w$", str(line))
#     if len(newList) > 0:
#         resof = newList[0].strip('A')
#         resof = float(resof)
#         values.append(resof)


In [121]:
ditch = []
for n in range(len(uni)):
    alpha_id = [atta["uniProtKBCrossReferences"][0]["id"] for atta in uni]
    ret = ({"AlphaFold Structure ID": alpha_id[n]})
    ret1 = list({ k:v for k, v in ret.items() if v.startswith("GO:")})
    for r in ret1:
        ret.pop(r)
    ditch.append(ret)

ditch

[{'AlphaFold Structure ID': 'O15648'},
 {'AlphaFold Structure ID': 'Q38B42'},
 {},
 {'AlphaFold Structure ID': 'Q9NJP9'},
 {},
 {},
 {},
 {},
 {},
 {'AlphaFold Structure ID': 'Q385E1'},
 {},
 {'AlphaFold Structure ID': 'Q38AC3'},
 {'AlphaFold Structure ID': 'Q387E8'},
 {},
 {'AlphaFold Structure ID': 'Q387X2'},
 {},
 {'AlphaFold Structure ID': 'Q385L6'},
 {'AlphaFold Structure ID': 'Q38EA4'},
 {},
 {'AlphaFold Structure ID': 'Q383C6'},
 {}]

In [122]:
# Creating a dataframe and inserting columns with the data obtained so far
df1 = pd.DataFrame()
df1['UniProt Accession'] = ids
df1['Gene Name'] = woohoo2
df1['Protein Name'] = woohoo
df1['Protein sequence length'] = seq_length
df1['Enzyme Number'] = pd.Series(ec_values)
df1['Subcellular locations'] = c_locss
df1

,UniProt Accession,Gene Name,Protein Name,Protein sequence length,Enzyme Number,Subcellular locations
0,O15648,pfk,ATP-dependent 6-phosphofructokinase,487,2.7.1.11,C:glycosome
1,Q38B42,Tb10.70.1370,Fructose-bisphosphate aldolase,372,4.1.2.13,C:cytosol
2,Q586Z6,Tb927.6.4300,Glyceraldehyde-3-phosphate dehydrogenase,402,1.2.1.12,"C:cytosol,C:glycosome"
3,Q9NJP9,GK,"Glycerol kinase, glycosomal",512,2.7.1.30,C:glycosome
4,Q583Z3,Tb927.4.3160,"Dihydroxyacetone phosphate acyltransferase, pu...",1242,2.3.1.42,"C:acidocalcisome,C:cytoplasm,C:glycosome,C:mit..."
5,Q57Y11,Tb927.4.4070,Mevalonate kinase,329,2.7.1.36,"C:cytosol,C:glycosome"
6,Q57ZS7,Tb927.5.2080,Inosine-5'-monophosphate dehydrogenase,491,1.1.1.205,"C:cytoplasm,C:glycosome,C:nucleolus"
7,Q586V2,Tb927.2.4210,Phosphoenolpyruvate carboxykinase (ATP),525,4.1.1.49,"C:contractile vacuole,C:cytosol,C:glycosome,C:..."
8,Q57YL8,Tb927.8.6170,Transketolase,669,2.2.1.1,"C:ciliary plasm,C:cytoplasm,C:cytosol,C:glycos..."
9,Q385E1,Tb11.02.4150,"Pyruvate, phosphate dikinase",913,2.7.9.1,C:glycosome


In [123]:
#Creating a dataframe out of a dictionary
df2 = pd.DataFrame.from_dict(ditch)
df2

,AlphaFold Structure ID
0,O15648
1,Q38B42
2,NaN
3,Q9NJP9
4,NaN
5,NaN
6,NaN
7,NaN
8,NaN
9,Q385E1


In [124]:
# Merging two dataframes using pd.concat
df_merged = pd.concat([df1, df2], axis=1)
df_merged

,UniProt Accession,Gene Name,Protein Name,Protein sequence length,Enzyme Number,Subcellular locations,AlphaFold Structure ID
0,O15648,pfk,ATP-dependent 6-phosphofructokinase,487,2.7.1.11,C:glycosome,O15648
1,Q38B42,Tb10.70.1370,Fructose-bisphosphate aldolase,372,4.1.2.13,C:cytosol,Q38B42
2,Q586Z6,Tb927.6.4300,Glyceraldehyde-3-phosphate dehydrogenase,402,1.2.1.12,"C:cytosol,C:glycosome",NaN
3,Q9NJP9,GK,"Glycerol kinase, glycosomal",512,2.7.1.30,C:glycosome,Q9NJP9
4,Q583Z3,Tb927.4.3160,"Dihydroxyacetone phosphate acyltransferase, pu...",1242,2.3.1.42,"C:acidocalcisome,C:cytoplasm,C:glycosome,C:mit...",NaN
5,Q57Y11,Tb927.4.4070,Mevalonate kinase,329,2.7.1.36,"C:cytosol,C:glycosome",NaN
6,Q57ZS7,Tb927.5.2080,Inosine-5'-monophosphate dehydrogenase,491,1.1.1.205,"C:cytoplasm,C:glycosome,C:nucleolus",NaN
7,Q586V2,Tb927.2.4210,Phosphoenolpyruvate carboxykinase (ATP),525,4.1.1.49,"C:contractile vacuole,C:cytosol,C:glycosome,C:...",NaN
8,Q57YL8,Tb927.8.6170,Transketolase,669,2.2.1.1,"C:ciliary plasm,C:cytoplasm,C:cytosol,C:glycos...",NaN
9,Q385E1,Tb11.02.4150,"Pyruvate, phosphate dikinase",913,2.7.9.1,C:glycosome,Q385E1


In [125]:
# To add None when there is no information available
df_merged.where(df_merged.notnull(), None)

,UniProt Accession,Gene Name,Protein Name,Protein sequence length,Enzyme Number,Subcellular locations,AlphaFold Structure ID
0,O15648,pfk,ATP-dependent 6-phosphofructokinase,487,2.7.1.11,C:glycosome,O15648
1,Q38B42,Tb10.70.1370,Fructose-bisphosphate aldolase,372,4.1.2.13,C:cytosol,Q38B42
2,Q586Z6,Tb927.6.4300,Glyceraldehyde-3-phosphate dehydrogenase,402,1.2.1.12,"C:cytosol,C:glycosome",None
3,Q9NJP9,GK,"Glycerol kinase, glycosomal",512,2.7.1.30,C:glycosome,Q9NJP9
4,Q583Z3,Tb927.4.3160,"Dihydroxyacetone phosphate acyltransferase, pu...",1242,2.3.1.42,"C:acidocalcisome,C:cytoplasm,C:glycosome,C:mit...",None
5,Q57Y11,Tb927.4.4070,Mevalonate kinase,329,2.7.1.36,"C:cytosol,C:glycosome",None
6,Q57ZS7,Tb927.5.2080,Inosine-5'-monophosphate dehydrogenase,491,1.1.1.205,"C:cytoplasm,C:glycosome,C:nucleolus",None
7,Q586V2,Tb927.2.4210,Phosphoenolpyruvate carboxykinase (ATP),525,4.1.1.49,"C:contractile vacuole,C:cytosol,C:glycosome,C:...",None
8,Q57YL8,Tb927.8.6170,Transketolase,669,2.2.1.1,"C:ciliary plasm,C:cytoplasm,C:cytosol,C:glycos...",None
9,Q385E1,Tb11.02.4150,"Pyruvate, phosphate dikinase",913,2.7.9.1,C:glycosome,Q385E1


In [126]:
# To write the dataframe as a csv file
df_merged.to_csv(un.output) #https://stackoverflow.com/questions/69501185/save-a-list-of-tuple-to-csv-file-in-python